## Modèle cible – Table `product_ec`

### Objectif
- Représenter la distribution d’un produit catalogue dans plusieurs entrepôts
- Servir de table volumineuse pour les traitements Spark
- Supporter les agrégations et le calcul de scores environnementaux

---

### Structure détaillée de la table

| Colonne | Type de données | Étendue / Domaine | Description |
|------|---------------|------------------|------------|
| `id_product_ec` | BIGINT / SERIAL | 1 → plusieurs millions | Identifiant technique unique de la ligne |
| `id_product_ref` | INT | 1 → 20 000 (ou +) | Référence produit catalogue |
| `size` | VARCHAR(5) | XS, S, M, L, XL… | Taille du produit |
| `id_warehouse` | INT | 1 → 100 | Identifiant de l’entrepôt |
| `stock_qty` | BIGINT | 0 → plusieurs millions | Quantité disponible simulée |
| `ec_process` | NUMERIC(12,4) | 0 → 1 000 (ordre de grandeur) | Impact environnemental du process |
| `ec_transport` | NUMERIC(12,4) | 0 → 1 000 (ordre de grandeur) | Impact environnemental du transport |
| `ec_total` | NUMERIC(12,4) | 0 → 2 000 | Somme de `ec_process` et `ec_transport` |

---

### Contraintes logiques envisagées
- Une ligne représente :
  - un produit
  - une taille
  - un entrepôt
- Unicité logique possible :
  - (`id_product_ref`, `size`, `id_warehouse`)
- `ec_total` dérivé de :
  - `ec_process + ec_transport`

---

### Hypothèses de volumétrie

- Nombre de produits catalogue (`R`) : 20 000 à 100 000
- Nombre moyen de tailles par produit (`T`) : 4
- Nombre d’entrepôts (`W`) : 100
- Proportion d’entrepôts réellement concernés (`p`) : 0,4

---

### Volumétrie estimée

- Formule :
  - `R × T × (W × p)`
- Exemple cible :
  - 20 000 × 4 × 40 ≈ **3,2 millions de lignes**
- Volumétrie extensible :
  - ajout de produits
  - ajout de tailles
  - ajout de périodes
  - concaténation de plusieurs sources

---

### Justification Big Data
- Table volontairement volumineuse
- Calculs distribués nécessaires :
  - agrégations par produit
  - calculs de moyenne
- Shuffle Spark induit par les groupBy
- Données exploitables par des modules analytiques ultérieurs